In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import collections
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import colors
import matplotlib.image as mpimg
from tqdm import tqdm
from typing import List, Tuple, Dict, Any
import gc

# Plot Statistics of Descriptor Evaluation using  Mutual Nearest Neighbour Matching (NN2W)


In [ ]:
#####################
### FUNCTIONS
#####################
def save_figure(
  path_output:str,
  fig_name:str, 
  figure: mpl.figure.Figure,
  dpi:int=1200,
  tight_layout:bool=False) -> None:

    if not os.path.exists(path_output):
        os.makedirs(path_output, exist_ok=True)

    f_out = os.path.join(path_output, fig_name)
  
    if tight_layout:
        figure.savefig(f_out, bbox_inches='tight', pad_inches=0, dpi=dpi)
    else:
        figure.savefig(f_out, dpi=dpi)


#####################
### SETTINGS
#####################
id2name = {
    'sift': 'SIFT',
    'lift': 'LIFT',
    'tilde': 'TILDE',
    'superpoint': 'SuperPoint',
    'tcovdet': 'TCovDet',
    'tfeat': 'TFeat',
    'doap': 'DOAP',
    'v_set_01': 'Set 1',
    'v_set_02': 'Set 2',
    'v_set_03': 'Set 3',
    'v_set_04': 'Set 4',
    'v_set_05': 'Set 5',
    'v_set_06': 'Set 6',
    'v_set_07': 'Set 7',
    'v_set_08': 'Set 8',
    'v_set_09': 'Set 9',
    'v_set_10': 'Set 10',
    'v_set_11': 'Set 11',
    'v_set_12': 'Set 12',
    'v_set_13': 'Set 13'
}

color_scheme = {
  'sift': '#1f77b4',
  'lift': '#ff7f0e',
  'tilde': '#2ca02c',
  'superpoint': '#d62728',
  'tcovdet': '#9467bd',
  'tfeat': '#2ca02c',
  'doap': '#9467bd'
}

keypoint_threshold_colors = ['skyblue', 'purple', 'crimson']

marker_scheme = {
  'sift': 'o',
  'lift': 'x',
  'tilde': 'd',
  'superpoint': '^',
  'tcovdet': 'h',
  'tfeat': '*',
  'doap': 'p'
}

line_styles = ['-', '-.', ':', '--']

data_dir = '/home/mizzade/Workspace/diplom/outputs/eval_matching_pipeline'
output_dir = '/home/mizzade/Workspace/diplom/outputs/eval_matching_pipeline/plots_nn2w'
file_name = 'descriptor_matching_eisert_nn2w_fmatrix.csv'


#####################
### MAIN
#####################

# Load data
df = pd.read_csv(os.path.join(data_dir, file_name), sep=',', comment='#')

descriptor_names = sorted(df.descriptor_name.unique())
detector_names = sorted(df.detector_name.unique())
set_names = sorted(df.set_name.unique())
desc_distance_thresholds = sorted(df.desc_distance_threshold.unique())
kpts_thresholds = sorted(df.kpts_threshold.unique())

desc_dist = desc_distance_thresholds[0]
kp_thresh = kpts_thresholds[0]

# Index(['collection_name', 'set_name', 'kpts_threshold', 'descriptor_name',
#        'detector_name', 'matching_method', 'desc_distance_threshold',
#        'max_num_matches', 'num_matches', 'matchability', 'accuracy'],
#       dtype='object')
# PART I

        



## Matchability and Accuracy
Für jede Kombination aus Deskriptor und Detektor finde die `Matchability`, `Accuracy` und deren `Score`.

- `Matchability` ist der Prozentsatz alle erfolreich zugeordneten Desrkiptoren.
- `Accuracy` gibt die durchschnittliche Genauigkeit bei Matches an.
- `Score` ist der Mittelwert der ersten beiden.

In [ ]:
df.head()

In [ ]:
kp_thresh = kpts_thresholds[0]

heatmap_accm = [] # accuracy of matches
heatmap_mat = []  # matchability

heatmap_acci = [] # accuracy of inliers
heatmap_ir = []   # inlier ratio

for descriptor_name in descriptor_names:
    
    accm_per_detector = [] # acc for matches
    mat_per_detector = []  # match ratio
    acci_per_detector = [] # acc for inliers
    ir_per_detector = []   # inlier ratio
    
    for detector_name in detector_names:
        
        _df = df[(df.kpts_threshold == kp_thresh) &
                 (df.desc_distance_threshold == desc_dist) &
                 (df.descriptor_name == descriptor_name) &
                 (df.detector_name == detector_name)]
        
        accm = np.mean(_df.accuracy_matches) # accuracy of matches
        mat = np.mean(_df.matchability)      # matchability ratio
        
        acci = np.mean(_df.accuracy_inliers) # accuracy of inliers
        ir = np.mean(_df.inlier_ratio)       # inlier ratio
        
        # Add to corresponding lists
        accm_per_detector.append(accm)
        mat_per_detector.append(mat)
        acci_per_detector.append(acci)
        ir_per_detector.append(ir)
        
    # Add to corresponding lists    
    heatmap_accm.append(accm_per_detector)
    heatmap_mat.append(mat_per_detector)
    heatmap_acci.append(acci_per_detector)
    heatmap_ir.append(ir_per_detector)

    # Convet to numpy array    
heatmap_accm = np.array(heatmap_accm)
heatmap_mat = np.array(heatmap_mat)
heatmap_acci = np.array(heatmap_acci)
heatmap_ir = np.array(heatmap_ir)

###############
### Drawing
###############
plt.rcParams['figure.figsize'] = [15, 12]
fig, axes = plt.subplots(2, 3)
cmap = mpl.colors.LinearSegmentedColormap.from_list("", ['tomato', 'gold', 'forestgreen'], 10)
text_color = 'black'

# Matchability
###################
ax = axes[0][0]
ax.tick_params(
    axis='both',         # changes apply to the axis
    which='both',        # both major and minor ticks are affected
    bottom=False,        # ticks along the bottom edge are off
    left=False)          # ticks along left edge are off

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax.set_xticklabels([''] + [id2name[x] for x in detector_names])
ax.set_yticklabels( [''] + [id2name[x] for x in descriptor_names])

ax.set_xlabel('Detektoren')
ax.set_ylabel('Deskriptoren')

ax.set_title('Übereinstimmungsquote')

_vals = heatmap_mat[~np.isnan(heatmap_mat)]
ax.imshow(heatmap_mat, cmap=cmap, vmin=np.min(_vals), vmax=np.max(_vals))

# Loop over data to dimensions and create text annotations.
for i in range(len(descriptor_names)):
    for j in range(len(detector_names)):
        
        _heatmap_mat = heatmap_mat * 100
        t = '' if np.isnan(heatmap_mat[i, j]) else '{:.1f}'.format(_heatmap_mat[i, j])
        text = ax.text(j, i, 
                       t, 
                       ha='center', 
                       va='center', 
                       color=text_color);

# Accuracy
###################
ax = axes[0][1]
ax.tick_params(
    axis='both',         # changes apply to the axis
    which='both',        # both major and minor ticks are affected
    bottom=False,        # ticks along the bottom edge are off
    left=False)          # ticks along left edge are off

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax.set_xticklabels([''] + detector_names)
ax.set_yticklabels([])
#ax.set_yticklabels( [''] + descriptor_names)

#ax.set_ylabel('Deskriptoren')
ax.set_xlabel('Detektoren')

ax.set_title('Genauigkeit der Matches')

_vals = heatmap_accm[~np.isnan(heatmap_accm)]
ax.imshow(heatmap_accm, cmap=cmap, vmin=np.min(_vals), vmax=np.max(_vals))

for i in range(len(descriptor_names)):
    for j in range(len(detector_names)):
        
        _heatmap_accm = heatmap_accm * 100
        t = '' if np.isnan(heatmap_accm[i, j]) else '{:.1f}'.format(_heatmap_accm[i, j])
        text = ax.text(j, i, 
                       t, 
                       ha='center', 
                       va='center', 
                       color=text_color);

# Remove plot
###################
ax = axes[0][2]
ax.axis('off')

# Inlier Ratio
###################
ax = axes[1][0]
ax.tick_params(
    axis='both',         # changes apply to the axis
    which='both',        # both major and minor ticks are affected
    bottom=False,        # ticks along the bottom edge are off
    left=False)          # ticks along left edge are off

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax.set_xticklabels([''] + [id2name[x] for x in detector_names])
ax.set_yticklabels([])
ax.set_yticklabels( [''] + [id2name[x] for x in descriptor_names])

ax.set_ylabel('Deskriptoren')
ax.set_xlabel('Detektoren')

ax.set_title('Inlierquote')

_vals = heatmap_ir[~np.isnan(heatmap_ir)]
ax.imshow(heatmap_ir, cmap=cmap, vmin=np.min(_vals), vmax=np.max(_vals))

for i in range(len(descriptor_names)):
    for j in range(len(detector_names)):
        
        _heatmap_ir = heatmap_ir * 100
        t = '' if np.isnan(heatmap_ir[i, j]) else '{:.1f}'.format(_heatmap_ir[i, j])
        text = ax.text(j, i, 
                       t, 
                       ha='center', 
                       va='center', 
                       color=text_color);
        
# Quality of Inliers
###################
ax = axes[1][1]
ax.tick_params(
    axis='both',         # changes apply to the axis
    which='both',        # both major and minor ticks are affected
    bottom=False,        # ticks along the bottom edge are off
    left=False)          # ticks along left edge are off

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax.set_xticklabels([''] + [id2name[x] for x in detector_names])
ax.set_yticklabels([])
#ax.set_yticklabels( [''] + descriptor_names)

#ax.set_ylabel('Deskriptoren')
ax.set_xlabel('Detektoren')

ax.set_title('Genauigkeit der Inlier')

_vals = heatmap_acci[~np.isnan(heatmap_acci)]
ax.imshow(heatmap_acci, cmap=cmap, vmin=np.min(_vals), vmax=np.max(_vals))

for i in range(len(descriptor_names)):
    for j in range(len(detector_names)):
        
        _heatmap_acci = heatmap_acci * 100
        t = '' if np.isnan(heatmap_acci[i, j]) else '{:.1f}'.format(_heatmap_acci[i, j])
        text = ax.text(j, i, 
                       t, 
                       ha='center', 
                       va='center', 
                       color=text_color);


# Score
###################
ax = axes[1][2]
ax.tick_params(
    axis='both',         # changes apply to the axis
    which='both',        # both major and minor ticks are affected
    bottom=False,        # ticks along the bottom edge are off
    left=False)          # ticks along left edge are off

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

ax.set_xticklabels([''] + [id2name[x] for x in detector_names])
ax.set_yticklabels([])
#ax.set_yticklabels( [''] + descriptor_names)

#ax.set_ylabel('Deskriptoren')
ax.set_xlabel('Detektoren')

ax.set_title('Wertung')


scoremap = 0.25 * (heatmap_mat + heatmap_accm + heatmap_ir + heatmap_acci)
_vals = scoremap[~np.isnan(scoremap)]
ax.imshow(scoremap, cmap=cmap, vmin=np.min(_vals), vmax=np.max(_vals))

for i in range(len(descriptor_names)):
    for j in range(len(detector_names)):
        
        _scoremap = scoremap * 100
        t = '' if np.isnan(scoremap[i, j]) else '{:.1f}'.format(_scoremap[i, j])
        text = ax.text(j, i, 
                       t, 
                       ha='center', 
                       va='center', 
                       color=text_color);

        
        
fig.suptitle('Analyse von Übereinstimmungsquote und Genauigkeit bei maximal {} Keypoints'.format(kp_thresh), fontsize=16)
fig.tight_layout()
fig.subplots_adjust(top=0.9)

        
# TODO - SAVE IMAGE

### Zeige die besten und schlechtesten Ergebnisse

In [ ]:
### 1. Build descriptor/detector matrix
combinations = []

for  descriptor_name in descriptor_names:
    for detector_name in detector_names:
        combinations.append((descriptor_name, detector_name))
        
combinations = np.array(combinations).reshape((len(descriptor_names), len(detector_names), 2))

### 2. Sortiere combinationen nach Score

# 2.1 Find all indices of nan values to filter them out
is_invalid = np.isnan(scoremap.reshape(1, -1))[0]

# 2.2 Sort indices by their corresponding values in scoremap
# in descending order
idx_sorted = scoremap.reshape(-1, ).argsort()[::-1]

# 2.x Find all indices with nan values and filter them out
is_invalid = np.isnan(scoremap.reshape(-1,)[idx_sorted])
idx_sorted = idx_sorted[~is_invalid]

# 2.3 Unravel indices back to shape of scoremap to get
# coordinates to access descriptor and detector names.
idx_sorted = np.unravel_index(idx_sorted, scoremap.shape)

# 2.4 Convert to Nx2 array (row_id, col_id)
idx_best = np.vstack(idx_sorted).T

### First vs Last

In [ ]:
### Take best and worst:
selection = [combinations[i, j] for i,j in [idx_best[0], idx_best[-1]]]

mat_per_combi = []
accm_per_combi = []
ir_per_combi = []
acci_per_combi = []

for (descriptor_name, detector_name) in selection:
    
    accm_per_set = []
    mat_per_set = []
    acci_per_set = []
    ir_per_set = []
    
    for set_name in set_names:
        _df = df[(df.kpts_threshold == kp_thresh) &
          (df.desc_distance_threshold == desc_dist) &
          (df.descriptor_name == descriptor_name) &
          (df.detector_name == detector_name) & 
          (df.set_name == set_name)]
        
        accm = np.mean(_df.accuracy_matches) # accuracy of matches
        mat = np.mean(_df.matchability)      # matchability ratio
        
        acci = np.mean(_df.accuracy_inliers) # accuracy of inliers
        ir = np.mean(_df.inlier_ratio)       # inlier ratio
        
        # Add to lists
        accm_per_set.append(accm)
        mat_per_set.append(mat)
        acci_per_set.append(acci)
        ir_per_set.append(ir)
     
    accm_per_combi.append(accm_per_set)
    mat_per_combi.append(mat_per_set)
    acci_per_combi.append(acci_per_set)
    ir_per_combi.append(ir_per_set)
        

xs = [id2name[x] for x in set_names]
ys = np.array([[mat_per_combi, accm_per_combi], [ir_per_combi, acci_per_combi]])
titles = [['Übereinstimmungsquote', 'Qualität der Übereinstimmung'], ['Inlierquote', 'Qualität der Inlier']]

### LINIEN GRAPHEN
fig, axes = plt.subplots(nrows=2, ncols=2)
for i in range(axes.shape[0]):
    for j in range(axes.shape[1]):
        ax = axes[i][j]
        ax.set_ylim(0, 1)
        ax.tick_params(axis='x', which='both', labelrotation=30) # Rotate x-axis label
        ax.set_title(titles[i][j])
        
        for k, (descriptor_name, detector_name) in enumerate(selection):
            
            # First make a line
            ax.plot(
                xs, 
                ys[i][j][k],
                linestyle=line_styles[2],
                alpha=0.3)
            
            # On Top make a scatter plot
            ax.scatter(
                xs,
                ys[i][j][k],
                marker=marker_scheme[list(marker_scheme.keys())[k]],
                label='{} mit {}-Keypoints'.format(id2name[descriptor_name], id2name[detector_name]))
            
        if (i, j) == (1, 0):
            ax.legend(loc='upper left', bbox_to_anchor=(0, -0.2))
            

### BALKEN GRAPHEN
fig, axes = plt.subplots(nrows=2, ncols=2)
for i in range(axes.shape[0]):
    for j in range(axes.shape[1]):
        ax = axes[i][j]
        ax.set_ylim(0, 1)
        ax.tick_params(axis='x', which='both', labelrotation=30) # Rotate x-axis label
        ax.set_title(titles[i][j])

        # get the first M colors of matplotlib standard color palette
        _colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:len(selection)]

        x_labels = xs
        N = len(x_labels)   # number of ticks
        M = len(selection)  # number of bars per tick
        
        # Tick position values
        ticks = np.arange(N)
        
        # Width of one bar:
        width_of_bar = 0.3
        
        # margin after each bar:
        margin_of_bar = 0.1
        
        # Set labels for each tick
        ax.set_xticklabels(x_labels)
        
        # Width of a "block" of bars.
        width_of_block = M * width_of_bar + (M-1) * margin_of_bar
        
        # Set position of ticks
        ax.set_xticks(ticks + width_of_block * 0.5 - width_of_bar * 0.5 - margin_of_bar)

        for num_of_tick, set_name in enumerate(set_names):
            for num_of_bar, (descriptor_name, detector_name) in enumerate(selection):
                _fcolor = list(colors.to_rgba(_colors[num_of_bar]))
                _fcolor[3] = 0.3 # set alpha to 0.3
                _ecolor = colors.to_rgba(_colors[num_of_bar])
                
                labeltext = None
                if num_of_tick == 0:
                    labeltext = '{} mit {}-Keypoints'.format(id2name[descriptor_name], id2name[detector_name])
                
                ax.bar(
                    ticks[num_of_tick] + num_of_bar * width_of_bar + (num_of_bar -1) * margin_of_bar,
                    ys[i][j][num_of_bar][num_of_tick],
                    width_of_bar,
                    facecolor=_fcolor,
                    edgecolor=_ecolor,
                    label=labeltext)
                
        if (i, j) == (1, 0):
            ax.legend(loc='upper left', bbox_to_anchor=(0, -0.2))

### TOP 5

In [ ]:
### Take top5:
selection = [combinations[i, j] for i,j in idx_best[:5]]

mat_per_combi = []
accm_per_combi = []
ir_per_combi = []
acci_per_combi = []

for (descriptor_name, detector_name) in selection:
    
    accm_per_set = []
    mat_per_set = []
    acci_per_set = []
    ir_per_set = []
    
    for set_name in set_names:
        _df = df[(df.kpts_threshold == kp_thresh) &
          (df.desc_distance_threshold == desc_dist) &
          (df.descriptor_name == descriptor_name) &
          (df.detector_name == detector_name) & 
          (df.set_name == set_name)]
        
        accm = np.mean(_df.accuracy_matches) # accuracy of matches
        mat = np.mean(_df.matchability)      # matchability ratio
        
        acci = np.mean(_df.accuracy_inliers) # accuracy of inliers
        ir = np.mean(_df.inlier_ratio)       # inlier ratio
        
        # Add to lists
        accm_per_set.append(accm)
        mat_per_set.append(mat)
        acci_per_set.append(acci)
        ir_per_set.append(ir)
     
    accm_per_combi.append(accm_per_set)
    mat_per_combi.append(mat_per_set)
    acci_per_combi.append(acci_per_set)
    ir_per_combi.append(ir_per_set)
        
xs = [id2name[x] for x in set_names]
ys = np.array([[mat_per_combi, accm_per_combi], [ir_per_combi, acci_per_combi]])
titles = [['Übereinstimmungsquote', 'Qualität der Übereinstimmung'], ['Inlierquote', 'Qualität der Inlier']]

### LINIEN GRAPHEN
fig, axes = plt.subplots(nrows=2, ncols=2)
for i in range(axes.shape[0]):
    for j in range(axes.shape[1]):
        ax = axes[i][j]
        ax.set_ylim(0, 1)
        ax.tick_params(axis='x', which='both', labelrotation=30) # Rotate x-axis label
        ax.set_title(titles[i][j])
        
        for k, (descriptor_name, detector_name) in enumerate(selection):
            # First make a line
            ax.plot(
                xs, 
                ys[i][j][k],
                linestyle=line_styles[2],
                alpha=0.5)
            
            # On Top make a scatter plot
            ax.scatter(
                xs,
                ys[i][j][k],
                marker=marker_scheme[list(marker_scheme.keys())[k]],
                label='{} mit {}-Keypoints'.format(id2name[descriptor_name], id2name[detector_name]))
            
        if (i, j) == (1, 0):
            ax.legend(loc='upper left', bbox_to_anchor=(0, -0.2))

fig, axes = plt.subplots(nrows=4, ncols=1)
axes = axes.reshape(2, 2)
for i in range(axes.shape[0]):
    for j in range(axes.shape[1]):
        ax = axes[i][j]
        ax.set_ylim(0, 1)
        ax.tick_params(axis='x', which='both', labelrotation=30) # Rotate x-axis label
        ax.set_title(titles[i][j])

        # get the first M colors
        _colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:len(selection)]

        x_labels = xs
        N = len(x_labels)   # number of ticks
        M = len(selection)  # number of bars per tick
        D = 3               # distance between ticks
        
        # Tick position values
        ticks = np.arange(N) * D
        
        # Width of one bar:
        width_of_bar = 0.5 * D / M
        
        # margin after each bar:
        margin_of_bar = 0.5 * width_of_bar
        
        # Set labels for each tick
        ax.set_xticklabels(x_labels)
        
        # Width of a "block" of bars.
        width_of_block = M * width_of_bar + (M-1) * margin_of_bar
        
        # Set position of ticks
        ax.set_xticks(ticks + width_of_block * 0.5 - width_of_bar * 0.5 - margin_of_bar)

        for num_of_tick, set_name in enumerate(set_names):
            for num_of_bar, (descriptor_name, detector_name) in enumerate(selection):
                _fcolor = list(colors.to_rgba(_colors[num_of_bar]))
                _fcolor[3] = 0.3 # set alpha to 0.3
                _ecolor = colors.to_rgba(_colors[num_of_bar])
                
                labeltext = None
                if num_of_tick == 0:
                    labeltext = '{} mit {}-Keypoints'.format(id2name[descriptor_name], id2name[detector_name])
                
                ax.bar(
                    ticks[num_of_tick] + num_of_bar * width_of_bar + (num_of_bar -1) * margin_of_bar,
                    ys[i][j][num_of_bar][num_of_tick],
                    width_of_bar,
                    facecolor=_fcolor,
                    edgecolor=_ecolor,
                    label=labeltext)
                
        if (i, j) == (0, 0):
            ax.legend(loc='lower right', bbox_to_anchor=(1.0, 1.01))
            
fig.tight_layout()
